In [1]:
import sys
sys.path.append('../src')

In [2]:
from gee_tools import Catalog, ZonalStats
import ee

In [3]:
ee.Initialize()

A shapefile can be called directly from GEE assets, or easily converted using gee_helpers.gpd_to_gee or the geemap package. (see other examples)

In [4]:
AOIs = ee.FeatureCollection('users/afche18/Ethiopia_AOI')

### Option A: Retrieve Zonal Stats directly   
Output: ee.FeatureCollection with results / dictionary  
(recommended for small areas / low n)

In [5]:
zs = ZonalStats(
    collection_id='LANDSAT/LC08/C01/T1_8DAY_NDVI', 
    target_features=AOIs, 
    statistic_type="mean",
    frequency="annual",
    temporal_stat="mean"
)

In [6]:
zs?

In [7]:
res = zs.runZonalStats()

In [8]:
res

In [9]:
res.getInfo()

{'type': 'FeatureCollection',
 'columns': {'2013_NDVI_mean': 'Float<-1.0, 1.0>',
  '2014_NDVI_mean': 'Float<-1.0, 1.0>',
  '2015_NDVI_mean': 'Float<-1.0, 1.0>',
  '2016_NDVI_mean': 'Float<-1.0, 1.0>',
  '2017_NDVI_mean': 'Float<-1.0, 1.0>',
  '2018_NDVI_mean': 'Float<-1.0, 1.0>',
  '2019_NDVI_mean': 'Float<-1.0, 1.0>',
  '2020_NDVI_mean': 'Float<-1.0, 1.0>',
  '2021_NDVI_mean': 'Float<-1.0, 1.0>',
  '2022_NDVI_mean': 'Float<-1.0, 1.0>',
  'Area': 'Float',
  'Id': 'Integer',
  'system:index': 'String'},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[42.06158510035763, 9.397823252053993],
      [42.06119314725495, 9.20601121183654],
      [42.18901118834056, 9.20584119448768],
      [42.316813505733144, 9.205625705724506],
      [42.3125304107434, 9.301530485292622],
      [42.30825425836041, 9.397435608019856],
      [42.184926941544006, 9.39765098188209],
      [42.06158510035763, 9.397823252053993]]]},
   'id': '00000000000000000000',
   'pr

Convert results to data frame

In [10]:
import pandas as pd

In [11]:
res = res.getInfo()
feats = res['features']
cols = [list(f['properties'].keys()) for f in feats][0]
ids = [f['id'] for f in feats]
keys = [list(f['properties'].keys()) for f in feats]
values = [list(f['properties'].values()) for f in feats]
df = pd.DataFrame(index=ids, columns=cols, data=values)

In [12]:
df.head()

,2013_NDVI_mean,2014_NDVI_mean,2015_NDVI_mean,2016_NDVI_mean,2017_NDVI_mean,2018_NDVI_mean,2019_NDVI_mean,2020_NDVI_mean,2021_NDVI_mean,2022_NDVI_mean,Area,Id
00000000000000000000,0.308395,0.328255,0.325812,0.299736,0.250651,0.269451,0.271191,0.306436,0.292027,None,586.656,0
00000000000000000001,0.561220,0.449039,0.491076,0.508028,0.413701,0.477646,0.503715,0.441869,0.478778,None,523.326,1


### Option B: Submit a task  
Output: csv table on Google Drive   
(recommended for big areas / high n)

In [13]:
zs = ZonalStats(
    collection_id='LANDSAT/LC08/C01/T1_8DAY_NDVI', 
    target_features=AOIs, 
    statistic_type="mean",
    frequency="annual",
    temporal_stat="mean",
    output_dir="pretty_folder",
    output_name="pretty_file"
)

In [14]:
zs.runZonalStats()

In [15]:
zs.task.status()

{'state': 'READY',
 'description': 'Zonal statistics for T1_8DAY_NDVI',
 'creation_timestamp_ms': 1644437897978,
 'update_timestamp_ms': 1644437897978,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_FEATURES',
 'id': 'OHEUNNUDAX4GKQUUPI5YI2WM',
 'name': 'projects/earthengine-legacy/operations/OHEUNNUDAX4GKQUUPI5YI2WM'}

In [17]:
zs.task.status()

{'state': 'COMPLETED',
 'description': 'Zonal statistics for T1_8DAY_NDVI',
 'creation_timestamp_ms': 1644437897978,
 'update_timestamp_ms': 1644437978321,
 'start_timestamp_ms': 1644437975749,
 'task_type': 'EXPORT_FEATURES',
 'destination_uris': ['https://drive.google.com/#folders/1csp0ZlY2eTPA2jgII2LoS3p1BQMFGlO5'],
 'attempt': 1,
 'id': 'OHEUNNUDAX4GKQUUPI5YI2WM',
 'name': 'projects/earthengine-legacy/operations/OHEUNNUDAX4GKQUUPI5YI2WM'}

In [18]:
zs.reportRunTime()

Completed
Runtime: 0 minutes and 2 seconds
